# Break: Reducible Polynomial Attack

**Module 02** | Breaking Weak Parameters

*Using a reducible polynomial instead of an irreducible one creates zero divisors and destroys field structure.*

## Why This Matters

In Module 02 we learned the fundamental construction:

$$\text{GF}(p^n) = \mathbb{F}_p[x] / (f(x))$$

This yields a **field** (every nonzero element invertible) if and only if $f(x)$ is
**irreducible** over $\mathbb{F}_p$.

But what if someone accidentally (or maliciously) uses a **reducible** polynomial?
The quotient ring is no longer a field. It has **zero divisors** -- nonzero elements
whose product is zero. Any cryptographic scheme that assumes field arithmetic
(division, inverses) will break catastrophically.

## The Scenario

Suppose we want to build a field with $5^2 = 25$ elements. The correct approach is:

$$\text{GF}(25) = \mathbb{F}_5[x] / (f(x))$$

where $f(x)$ is an irreducible degree-2 polynomial over $\mathbb{F}_5$.

But someone picks $f(x) = x^2 + 3x + 2$ without checking irreducibility.

**Spoiler**: Over $\mathbb{F}_5$, this polynomial factors as $(x + 1)(x + 2)$.
The quotient ring has **zero divisors** and is NOT a field.

In [ ]:
# === Step 1: Show that x^2 + 3x + 2 factors over GF(5) ===
R.<x> = PolynomialRing(GF(5))

f = x^2 + 3*x + 2
print(f'f(x) = {f}')
print(f'Is f(x) irreducible over GF(5)? {f.is_irreducible()}')
print(f'Factorization: {f.factor()}')
print()

# Check roots explicitly
for a in GF(5):
    val = f(a)
    root_marker = '  <-- ROOT!' if val == 0 else ''
    print(f'  f({a}) = {val}{root_marker}')

print()
print('f(x) has roots at x = 4 (= -1 mod 5) and x = 3 (= -2 mod 5).')
print('So f(x) = (x + 1)(x + 2) = (x - 4)(x - 3) over GF(5).')

## Step 2: Find Zero Divisors

Build the quotient ring $\mathbb{F}_5[x]/(x^2 + 3x + 2)$ and find elements whose
product is zero even though neither element is zero.

Since $f(x) = (x+1)(x+2)$, the images of $(x+1)$ and $(x+2)$ in the quotient ring
should multiply to zero (because their product equals $f(x) \equiv 0$ in the quotient).

In [ ]:
# === Step 2: Zero divisors in the quotient ring ===
R.<x> = PolynomialRing(GF(5))
f = x^2 + 3*x + 2

S.<a> = R.quotient(f)
print(f'Quotient ring: F_5[x] / ({f})')
print(f'Elements: {list(S)}')
print(f'Number of elements: {len(list(S))}')
print()

# The zero divisors: (a + 1) and (a + 2)
zd1 = a + 1
zd2 = a + 2
product = zd1 * zd2

print(f'(a + 1) = {zd1}  -- is this zero? {zd1 == S.zero()}')
print(f'(a + 2) = {zd2}  -- is this zero? {zd2 == S.zero()}')
print(f'(a + 1) * (a + 2) = {product}  -- ZERO!')
print()
print('Two nonzero elements multiply to zero. This ring has ZERO DIVISORS.')
print('It is NOT a field.')

In [ ]:
# === Find ALL zero divisors systematically ===
R.<x> = PolynomialRing(GF(5))
S.<a> = R.quotient(x^2 + 3*x + 2)
elems = list(S)

print('All zero-divisor pairs (nonzero elements whose product is 0):')
print()
zero_divisors = set()
for i, e1 in enumerate(elems):
    for e2 in elems[i:]:
        if e1 != S.zero() and e2 != S.zero() and e1 * e2 == S.zero():
            print(f'  {e1} * {e2} = 0')
            zero_divisors.add(e1)
            zero_divisors.add(e2)

print(f'\nZero divisors: {sorted(zero_divisors, key=str)}')
print(f'Count: {len(zero_divisors)} out of {len(elems) - 1} nonzero elements')

## Step 3: How Zero Divisors Break a Toy Scheme

Consider a toy "encryption" scheme in the quotient ring:

- **Key**: a nonzero element $k$ in the ring
- **Encrypt**: $c = m \cdot k$ (multiply message by key)
- **Decrypt**: $m = c \cdot k^{-1}$ (multiply ciphertext by inverse of key)

This works perfectly in a **field** (every nonzero $k$ has an inverse).
But if $k$ is a zero divisor, $k^{-1}$ does not exist, and decryption is impossible.
Worse: distinct messages can encrypt to the same ciphertext (information loss).

In [ ]:
# === Toy encryption scheme: works in a field, breaks with zero divisors ===
R.<x> = PolynomialRing(GF(5))
S.<a> = R.quotient(x^2 + 3*x + 2)

# Key is a zero divisor
key = a + 1
print(f'Key k = {key} (a zero divisor!)')
print()

# Try to encrypt several messages
print('Encrypting all nonzero messages with c = m * k:')
ciphertexts = {}
for m in S:
    if m == S.zero():
        continue
    c = m * key
    print(f'  Encrypt({m}) = {m} * {key} = {c}')
    if str(c) not in ciphertexts:
        ciphertexts[str(c)] = []
    ciphertexts[str(c)].append(str(m))

print()
print('=== Collisions ===')
for c_val, msgs in ciphertexts.items():
    if len(msgs) > 1:
        print(f'  Ciphertext {c_val} came from messages: {msgs}')
        print(f'  --> Decryption is AMBIGUOUS! Information is lost.')

print()
print('Now try to compute k^(-1)...')
try:
    key_inv = key^(-1)
    print(f'  k^(-1) = {key_inv}')
except Exception as e:
    print(f'  ERROR: {e}')
    print(f'  The inverse does not exist! Decryption is impossible.')

In [ ]:
# === Contrast: the same scheme with an IRREDUCIBLE polynomial works perfectly ===
R.<x> = PolynomialRing(GF(5))

# x^2 + 2 is irreducible over GF(5) (check: no roots)
g = x^2 + 2
print(f'g(x) = {g}')
print(f'Irreducible? {g.is_irreducible()}')

T.<b> = R.quotient(g)
print(f'This quotient ring IS a field: {T.is_field()}')
print()

# Same key value, but now in a field
key_good = b + 1
print(f'Key k = {key_good}')
key_good_inv = key_good^(-1)
print(f'k^(-1) = {key_good_inv}')
print(f'k * k^(-1) = {key_good * key_good_inv}  (= 1, as expected)')
print()

# Encrypt and decrypt a message
msg = 3*b + 4
ct = msg * key_good
recovered = ct * key_good_inv
print(f'Message:     m = {msg}')
print(f'Ciphertext:  c = m * k = {ct}')
print(f'Decrypted:   c * k^(-1) = {recovered}')
print(f'Correct?     {recovered == msg}')

## The Fix: Always Verify Irreducibility

Before using a polynomial $f(x)$ to build a quotient ring for cryptographic purposes:

1. **Check irreducibility**: Use `f.is_irreducible()` or test for roots (sufficient for degree 2 or 3).
2. **Use known irreducible polynomials**: Standards like AES specify the exact irreducible polynomial.
3. **Never trust user-supplied polynomials** without verification.

| Property | Irreducible $f(x)$ | Reducible $f(x)$ |
|---|---|---|
| Quotient ring $\mathbb{F}_p[x]/(f(x))$ | **Field** | Not a field |
| Zero divisors | **None** | Present |
| Every nonzero element invertible | **Yes** | No |
| Safe for crypto (inversion, division) | **Yes** | **No** -- breaks schemes |

## Exercises

1. **Another reducible polynomial**: Try $f(x) = x^2 + 4x + 3$ over $\mathbb{F}_5$.
   Factor it, build the quotient ring, and find all zero divisors.

2. **Degree 3**: Over $\mathbb{F}_2$, the polynomial $x^3 + x^2 + x + 1 = (x+1)(x^2+1) = (x+1)^3$
   is reducible. Build $\mathbb{F}_2[x]/(x^3 + x^2 + x + 1)$ and find zero divisors.

3. **Counting zero divisors**: In the ring $\mathbb{F}_5[x]/(x^2 + 3x + 2)$, how many
   of the 25 elements are zero divisors? How many are units (invertible)?
   *Hint*: By the Chinese Remainder Theorem for rings,
   $\mathbb{F}_5[x]/((x+1)(x+2)) \cong \mathbb{F}_5 \times \mathbb{F}_5$.

4. **The analogy with integers**: We know $\mathbb{Z}/15\mathbb{Z}$ has zero divisors
   because $15 = 3 \times 5$. The analogue here is that $f(x)$ factors.
   Write down the complete parallel between $\mathbb{Z}/15\mathbb{Z}$ and
   $\mathbb{F}_5[x]/(x^2+3x+2)$.

In [ ]:
# === Exercise workspace ===
R.<x> = PolynomialRing(GF(5))

# Exercise 1: Try f(x) = x^2 + 4x + 3 over GF(5)
f_ex1 = x^2 + 4*x + 3
print(f'Exercise 1: f(x) = {f_ex1}')
print(f'Irreducible? {f_ex1.is_irreducible()}')
print(f'Factorization: {f_ex1.factor()}')
print()

# TODO: Build the quotient ring and find zero divisors
# S.<a> = R.quotient(f_ex1)
# ...

## Summary

| | Irreducible polynomial | Reducible polynomial |
|---|---|---|
| **Example** | $x^2 + 2$ over $\mathbb{F}_5$ | $x^2 + 3x + 2 = (x+1)(x+2)$ over $\mathbb{F}_5$ |
| **Quotient ring** | $\text{GF}(25)$ -- a field | $\mathbb{F}_5 \times \mathbb{F}_5$ -- NOT a field |
| **Zero divisors** | None | $(x+1)$ and $(x+2)$ multiply to $0$ |
| **Inverses** | Every nonzero element invertible | Some elements have no inverse |
| **Crypto safety** | Safe | **Broken** |

**Key takeaways:**
- A quotient ring $\mathbb{F}_p[x]/(f(x))$ is a field **only** when $f(x)$ is irreducible.
- If $f(x) = g(x) \cdot h(x)$, then $g(x)$ and $h(x)$ become zero divisors in the quotient.
- Zero divisors break any scheme that relies on multiplicative inverses (encryption, signatures, key exchange).
- **Always verify irreducibility** before using a polynomial to construct a field.
- This is the polynomial analogue of the integer fact: $\mathbb{Z}/n\mathbb{Z}$ is a field only when $n$ is prime.

---

*Back to [Module 02: Rings, Fields, and Polynomials](../README.md)*